# knn algorithm from scratch

In [2]:
import numpy as np
import urllib
from urllib.request import urlopen
import pandas as pd
import time

In [3]:
#helper function to set up the dataset
def read_file(filename):
    temping = []
    with open(filename) as f:
        for line in f:
            temp = list(map(int, line.split()))
            temping.append(temp)
    return np.array(temping)

def split_label(x):
    temping = []
    temping2 = []
    for i in x:
        temping.append(i[:-1])
        temping2.append(i[-1])
    return np.array(temping),np.array(temping2)

In [5]:
#read the train set, test set, validation set.
train_data = read_file('train_set.txt')
test_data = read_file('test_set.txt')
validation_data = read_file('validate_set.txt')

In [6]:
x_train,y_train = split_label(train_data)
x_valid,y_valid = split_label(validation_data)
x_test,y_test = split_label(test_data)

In [7]:
def find_distance(a,b):
    diff = np.array(a)-np.array(b)
    return np.sqrt(np.sum(np.square(diff)))

In [8]:
from scipy import stats

In [13]:
def knn(training,y_training,testing,k):
    st = time.time()
    predicted_valid = []
    for i in range(len(testing)):
        d  = []
        for j in range(len(training)):
            #find the distance between points
            d.append(find_distance(testing[i],training[j]))
        neighbor_index = np.array(d).argsort()[:k] #find the closest N points
        neighbor_result=[]
        for index in neighbor_index:
            neighbor_result.append(y_training[index])
        neighbor_result=np.array(neighbor_result)
        if len(stats.mode(neighbor_result).mode)==1:
            predict = stats.mode(neighbor_result).mode[0]      
        else:
            predict = np.random.choice(stats.mode(neighbor_result).mode)
        predicted_valid.append(predict)
    print(" %s seconds time" % (time.time() - st))
    return predicted_valid

For k = 1, 5, 9 and 15, build k-nearest neighbor classifiers from the training data. For each of these
values of k, write down a table of training errors (error on the training data) and the validation errors
(error on the validation data).

In [14]:
list_of_k = [1,5,9,15]
training_error_rate = []
valid_error_rate = []
test_error_rate = []
for i in list_of_k:
    training_predicted = knn(x_train,y_train,x_train,i)
    training_error_rate.append(1-(sum(training_predicted == y_train)/len(y_train)))
    valid_predicted = knn(x_train,y_train,x_valid,i)
    valid_error_rate.append(1-(sum(valid_predicted == y_valid)/len(y_valid)))
    test_predicted = knn(x_train,y_train,x_test,i)
    test_error_rate.append(1-(sum(test_predicted == y_test)/len(y_test)))

 39.276570558547974 seconds time
 19.632791996002197 seconds time
 19.666377305984497 seconds time
 38.72052359580994 seconds time
 19.63158130645752 seconds time
 20.23234534263611 seconds time
 39.767133712768555 seconds time
 19.676707983016968 seconds time
 19.69752335548401 seconds time
 38.55005645751953 seconds time
 19.63233780860901 seconds time
 19.47134757041931 seconds time


In [15]:
table1 = pd.DataFrame({'k' : list_of_k,
                        'training error' : training_error_rate,
                        'validation error' : valid_error_rate,
                      'test error':test_error_rate})

In [16]:
table1

,k,training error,validation error,test error
0,1,0.0000,0.082,0.094
1,5,0.0565,0.095,0.098
2,9,0.0685,0.104,0.101
3,15,0.0925,0.108,0.114


Using a projection as a pre-processing step affects the accuracy and
running-time of nearest neighbor classification.
This file represents a projection matrix P with 784 rows and 20 columns. Each column is a 784-dimensional unit vector, and the columns are orthogonal to each other.
Project the training, validation and test data onto the column space of this matrix.

In [17]:
def read_file_projection(filename):
    temping = []
    with open(filename) as f:
        for line in f:
            temp = list(map(float,line.split()))
            temping.append(temp)
    return np.array(temping)

In [18]:
projection_data = read_file_projection('projection.txt')

In [19]:
train_projected_matrix = np.matmul(x_train,projection_data)
valid_projected_matrix = np.matmul(x_valid,projection_data)
test_projected_matrix = np.matmul(x_test,projection_data)

In [20]:
list_of_k = [1,5,9,15]
training_error_rate = []
valid_error_rate = []
test_error_rate = []
for i in list_of_k:
    training_predicted = knn(train_projected_matrix,y_train,train_projected_matrix,i)
    training_error_rate.append(1-(sum(training_predicted == y_train)/len(y_train)))
    valid_predicted = knn(train_projected_matrix,y_train,valid_projected_matrix,i)
    valid_error_rate.append(1-(sum(valid_predicted == y_valid)/len(y_valid)))
    test_predicted = knn(train_projected_matrix,y_train,test_projected_matrix,i)
    test_error_rate.append(1-(sum(test_predicted == y_test)/len(y_test)))

 31.276735544204712 seconds time
 15.567809581756592 seconds time
 15.532227754592896 seconds time
 30.99457550048828 seconds time
 15.786275625228882 seconds time
 15.743521928787231 seconds time
 30.928754806518555 seconds time
 15.590014457702637 seconds time
 15.721471548080444 seconds time
 31.444502353668213 seconds time
 16.034120798110962 seconds time
 15.851611852645874 seconds time


In [21]:
table2 = pd.DataFrame({'k' : list_of_k,
                                'training error' : training_error_rate,
                                'validation error' : valid_error_rate,
                      'test error':test_error_rate})

In [22]:
table2

,k,training error,validation error,test error
0,1,0.0000,0.320,0.314
1,5,0.1945,0.299,0.301
2,9,0.2305,0.302,0.293
3,15,0.2570,0.289,0.296


The classification accuracy decrease a bit after using projection. However, the running time of my program increases